In [5]:
import numpy as np
from src._petab.CRP2_CPE.scripts import (
    create_CRP2_CPE_conditions,
    create_CRP2_CPE_Amici,
    create_CRP2_CPE_ODE,
    # create_CRP2_CPE_default_params,
)
from src.utils.params import ParameterContainer
from src.models.cpe import CPE, create_model
import src.utils.petab as pet

from src.models.sbml import CRP2_CPE
from src.models import cpe
import src.models.amici as am
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Define SBML/AMICI Model

In [6]:
observables_df = pet.define_observables({
    'xA': 'xA',
    'xB': 'xB',  
}, noise_value=0.02)

# Define a set of parameters to sweep (e.g. irreversible params, all params, base set of params, extended set, etc.)
pc = ParameterContainer.from_json("/PolyPESTO/src/data/parameters/CRP2_CPE.json")
pg = pc.get_parameter_group("IRREVERSIBLE")
# pg = pc.combine_groups(["IRREVERSIBLE", "REVERSIBLE"], "ALL")

# Define a set of conditions to generate synthetic data
t_eval = list(np.arange(0, 1, 0.1, dtype=float))
fA0s = np.array([0.25, 0.5, 0.75, 0.1], dtype=float)
cM0s = np.array([1.0, 1.0, 1.0, 1.0], dtype=float)
cond_df = create_CRP2_CPE_conditions(fA0s, cM0s)

# Create the model
model1 = create_CRP2_CPE_Amici(model_dir=ds_dir, force_compile=False)
model2 = create_CRP2_CPE_ODE()

# Generate and save the dataset
ds1 = model1.generate_dataset(
    param_group=pg, t_eval=t_eval, cond_df=cond_df, name=ds_name + "_amici"
)
ds2 = model2.generate_dataset(
    param_group=pg, t_eval=t_eval, cond_df=cond_df, name=ds_name + "_ode"
)

Creating SBML model (CRP2_CPE).
Creating CRP parameters.
                 filename : /PolyPESTO/src/data/datasets/CRP2_CPE/ds_0/CRP2_CPE.xml
         file size (byte) : 14278
           read time (ms) : 5.360126
        c-check time (ms) : 2.191544
      validation error(s) : 0
    (consistency error(s)): 0
    validation warning(s) : 0
  (consistency warning(s)): 0


ValueError: module_path '/PolyPESTO/amici_models/CRP2_CPE' is not a directory.

In [ ]:
# Define model parameters
true_params = {
    # 'kpAA': 1.0,
    "rA": 5.0,
    "rB": 1.0,
    "rX": 1.0,
    "KAA": 0.0,
    "KAB": 0.0,
    "KBA": 0.0,
    "KBB": 0.0,
}
model = am.set_model_parameters(model, true_params)

# Define conditions
fA0 = np.array([0.25, 0.5, 0.75], dtype=float)
cM0 = np.array([1.0, 1.0, 1.0], dtype=float)
A0 = fA0 * cM0
B0 = (1 - fA0) * cM0

# Define conditions dataframe
conditions_df = pet.define_conditions(
    init_conditions={
        "A0": list(A0),
        "B0": list(B0),
    }
)
print(conditions_df)

# Define conversion values
timepoints = list(np.arange(0, 1, 0.1, dtype=float))

amici_measurements_df, rdatas = am.define_measurements_amici(
    amici_model=model,
    timepoints=timepoints,
    conditions_df=conditions_df,
    observables_df=observables_df,
    obs_sigma=0.0,
    meas_sigma=0.005,
    debug_return_rdatas=True,
)

amici_measurements_df

### Define CPE Measurements

In [ ]:
cpe_model = cpe.Model.from_ratios(list(true_params.values()))
cpe_model.solve(fA0=0.75, M0=1.0, t_eval=timepoints, approach='wittmer')

# Not implemented yet!
cpe_measurements_df = cpe.define_measurements_cpe(
    cpe_model=cpe_model,
    timepoints=timepoints,
    conditions_df=conditions_df,
    observables_df=observables_df,
    obs_sigma=0.0,
    meas_sigma=0.005,
    approach="izu",
)

cpe_measurements_df